<a href="https://colab.research.google.com/github/yuki-tamaribuchi/ml_with_spotify_api/blob/master/spotify_cnn_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_pickle('/content/drive/MyDrive/spotify_df/j-rock-df_20210219_01')

In [3]:
cens_data=np.array([cens for cens in df['Cens']])

In [4]:
reshaped_cens_data=[]

for cens in cens_data:
  length=cens.shape[1]
  reshaped_cens_data.append(cens.reshape(length,12))

reshaped_cens_data=np.array(reshaped_cens_data)

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
artist_label=LabelEncoder().fit_transform(df['Artist'])

In [7]:
X_train,X_test,y_train,y_test=train_test_split(reshaped_cens_data,artist_label,random_state=0)

In [8]:
inputs=layers.Input(shape=(2584,12))
x=layers.Conv1D(16,3,padding='same',activation='relu')(inputs)
x=layers.MaxPool1D()(x)
x=layers.Conv1D(32,3,padding='same',activation='relu')(x)
x=layers.MaxPool1D()(x)
x=layers.Flatten()(x)
x=layers.Dense(3)(x)
outputs=layers.Softmax()(x)

In [9]:
model=tf.keras.Model(inputs=inputs,outputs=outputs)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2584, 12)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 2584, 16)          592       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1292, 16)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1292, 32)          1568      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 646, 32)           0         
_________________________________________________________________
flatten (Flatten)            (None, 20672)             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 62019 

In [11]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
hist=model.fit(X_train,y_train,epochs=5)

Epoch 1/5
4/4 [==============================] - 1s 68ms/step - loss: 1.3106 - accuracy: 0.2804
Epoch 2/5
4/4 [==============================] - 0s 63ms/step - loss: 1.1751 - accuracy: 0.4119
Epoch 3/5
4/4 [==============================] - 0s 67ms/step - loss: 1.1164 - accuracy: 0.5370
Epoch 4/5
4/4 [==============================] - 0s 63ms/step - loss: 1.0156 - accuracy: 0.5966
Epoch 5/5
4/4 [==============================] - 0s 64ms/step - loss: 0.9864 - accuracy: 0.5005


In [13]:
model.evaluate(X_test,y_test)

2/2 [==============================] - 0s 11ms/step - loss: 1.0400 - accuracy: 0.5500


[1.0400205850601196, 0.550000011920929]